In [1]:
from calculateColumnsForModel import numberToRoleName
import pandas as pd

In [19]:


pre_make_pivot_table_df = pd.read_excel("../data/pre_make_pivot_table_draft5.xlsx", dtype = {"gameId":"str"})
pd.set_option("display.max_columns", None)

In [17]:
fromvalue_pivot = pd.pivot_table(
    pre_make_pivot_table_df,
    values = "formvalue",
    index = ["gameId"],
    columns = ["side", "role", "elements"]
).reset_index()
fromvalue_pivot.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in fromvalue_pivot.columns]
fromvalue_pivot.rename(columns = {"gameId__" : "gameId"}, inplace = True)
for i in range(5):
    fromvalue_pivot.drop([f"Blue_{numberToRoleName(i)}_championDamageShare", f"Red_{numberToRoleName(i)}_championDamageShare"], axis=1, inplace=True)   # championdamageshare 다 지우고
    fromvalue_pivot[f"{numberToRoleName(i)}_wardsScorePerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_wardsScorePerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_wardsScorePerTime"]
    fromvalue_pivot.drop([f"Blue_{numberToRoleName(i)}_wardsScorePerTime", f"Red_{numberToRoleName(i)}_wardsScorePerTime"], axis=1, inplace=True)

fromvalue_pivot.drop(["Blue_Spt_kda", f"Red_Spt_kda"], axis=1, inplace=True)
fromvalue_pivot.drop(["Blue_Spt_killsPerTime", f"Red_Spt_killsPerTime"], axis=1, inplace=True)
fromvalue_pivot.drop(["Blue_Spt_deathsPerTime", f"Red_Spt_deathsPerTime"], axis=1, inplace=True)
fromvalue_pivot["Spt_assistsPerTimeDiff"] = fromvalue_pivot[f"Blue_Spt_assistsPerTime"] - fromvalue_pivot[f"Red_Spt_assistsPerTime"]
fromvalue_pivot.drop(["Blue_Spt_creepScorePerTime", f"Red_Spt_creepScorePerTime"], axis=1, inplace=True)
fromvalue_pivot.drop(["Blue_Spt_goldEarnedPerTime", f"Red_Spt_goldEarnedPerTime"], axis=1, inplace=True)
for i in range(4):
    
    fromvalue_pivot[f"{numberToRoleName(i)}_kdaDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_kda"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_kda"]
    #fromvalue_pivot[f"{numberToRoleName(i)}_championDamageShareDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_championDamageShare"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_championDamageShare"]
    fromvalue_pivot[f"{numberToRoleName(i)}_killsPerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_killsPerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_killsPerTime"]
    fromvalue_pivot[f"{numberToRoleName(i)}_deathsPerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_deathsPerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_deathsPerTime"]
    fromvalue_pivot[f"{numberToRoleName(i)}_assistsPerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_assistsPerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_assistsPerTime"]
    fromvalue_pivot[f"{numberToRoleName(i)}_creepScorePerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_creepScorePerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_creepScorePerTime"]
    
    fromvalue_pivot[f"{numberToRoleName(i)}_goldEarnedPerTimeDiff"] = fromvalue_pivot[f"Blue_{numberToRoleName(i)}_goldEarnedPerTime"] - fromvalue_pivot[f"Red_{numberToRoleName(i)}_goldEarnedPerTime"]
    fromvalue_pivot.drop([f"Blue_{numberToRoleName(i)}_kda", f"Red_{numberToRoleName(i)}_kda"], axis=1, inplace=True)
    fromvalue_pivot.drop([f"Blue_{numberToRoleName(i)}_creepScorePerTime", f"Red_{numberToRoleName(i)}_creepScorePerTime"], axis=1, inplace=True)
    fromvalue_pivot.drop([f"Blue_{numberToRoleName(i)}_goldEarnedPerTime", f"Red_{numberToRoleName(i)}_goldEarnedPerTime"], axis=1, inplace=True)

teamwinrate_df = pre_make_pivot_table_df[["gameId", "side", "teamWinrate"]].drop_duplicates()

teamwinrate_df = teamwinrate_df.pivot(index='gameId', columns='side', values=['teamWinrate', "teamGoldDiff", "teamKillDiff"])

# Subtract Red from Blue to get the new column
teamwinrate_df['teamWinrateDiff'] = teamwinrate_df['Blue'] - teamwinrate_df['Red']      # 이부분 고쳐야 함
#teamwinrate_df.drop(["Blue", "Red"], axis=1, inplace=True)
teamwinrate_df.rename(columns = {"Blue" : "Blue_Winrate", "Red": "Red_Winrate"}, inplace=True)


# Reset index to turn gameId back into a column
teamwinrate_diff_df = teamwinrate_df.reset_index()
headtohead_and_winner_df = pre_make_pivot_table_df[["gameId", "headtoHeadWinrate", "headtoHeadGoldDiff", "headtoHeadKillDiff", "winner"]].drop_duplicates()
final_pivot_table = fromvalue_pivot.merge(teamwinrate_diff_df, on = "gameId", how = "left")
final_pivot_table = final_pivot_table.merge(headtohead_and_winner_df, on = "gameId", how = "left")
#final_pivot_table.to_excel("../data/dataset.xlsx")
final_pivot_table.head()

,gameId,Top_wardsScorePerTimeDiff,Jgl_wardsScorePerTimeDiff,Mid_wardsScorePerTimeDiff,Adc_wardsScorePerTimeDiff,Spt_wardsScorePerTimeDiff,Top_kdaDiff,Top_creepScorePerTimeDiff,Top_goldEarnedPerTimeDiff,Jgl_kdaDiff,Jgl_creepScorePerTimeDiff,Jgl_goldEarnedPerTimeDiff,Mid_kdaDiff,Mid_creepScorePerTimeDiff,Mid_goldEarnedPerTimeDiff,Adc_kdaDiff,Adc_creepScorePerTimeDiff,Adc_goldEarnedPerTimeDiff,Blue_Winrate,Red_Winrate,teamWinrateDiff,headtoHeadWinrate,winner
0,107405837336900396,0.000248,-0.003559,-0.002301,-0.000993,-0.006697,-0.155000,0.001792,-0.320654,0.832500,-0.001424,0.037339,-1.680000,-0.004772,-0.426365,0.749524,0.007231,0.047862,0.333333,0.333333,0.000000,0.666667,Red
1,107405837336900398,0.000976,0.005070,0.000237,-0.000947,-0.003603,1.445556,-0.010306,-0.268639,0.978333,0.000088,0.490208,-1.568333,0.000400,-0.254619,3.017778,-0.008820,0.081886,0.200000,0.333333,-0.133333,0.500000,Blue
2,107405837336900400,-0.001793,-0.002274,0.000837,0.002205,0.005175,-3.454167,-0.007125,-0.583221,-4.214583,-0.005590,-0.911711,-6.212917,-0.013887,-0.935413,-1.870833,0.000687,-0.782814,0.300000,0.700000,-0.400000,0.500000,Red
3,107405837336900402,-0.000034,0.004542,0.002569,0.000605,0.003793,1.931667,-0.020408,0.157579,0.808333,0.000615,0.504056,2.398333,-0.006775,0.340173,1.566667,0.013198,0.664361,0.555556,0.333333,0.222222,0.500000,Blue
4,107405837336900404,0.000000,0.000270,0.001929,0.000008,0.000194,0.000000,0.000000,0.000000,-0.047024,0.005841,0.664337,-0.011667,-0.005086,-0.179443,-0.001190,-0.009291,-0.293392,0.333333,0.444444,-0.111111,0.250000,Blue


In [18]:
final_pivot_table.to_excel("../data/dataset_draft5_1.xlsx")